# Topless AlexNet 
The classic AlexNet that won the 2012 ImageNet competition canonically has 5 convolutional plus 3 fully connected layers. However, the fully connected layers (or tops) on many classic conv-nets may actually be more of a fashionable legacy and we can often get away with average pooling instead of used a dense layer(s) on top.  

https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks
https://karpathy.github.io/2019/04/25/recipe/ - Karpathy mentions the topless trend in conv-nets under the regularization tips section

In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import tensorflow as tf
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, SpatialDropout2D, Conv2D, MaxPooling2D, AveragePooling1D, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

import matplotlib.pyplot as plt

In [6]:
x_train = np.load('./data/train_x.npy')
y_train = np.load('./data/train_y.npy')
x_train.shape, y_train.shape, np.max(x_train)

((8192, 32, 32, 1), (8192, 1), 1)

In [13]:
# set up training data and labels
dim_x = 32
dim_y = 32
batch_size=32

# read in data/labels
print("data shapes", x_train.shape, y_train.shape, "classes: ",len(np.unique(y_train)))

classes = len(np.unique(y_train))
x_train = x_train.reshape((-1, dim_x,dim_y,1))
# convert labels to one-hot
print(np.unique(y_train))
y = np.zeros((np.shape(y_train)[0],len(np.unique(y_train))))

# convert index labels to one-hot
for ii in range(len(y_train)):
    #print(y_train[ii])
    y[ii,y_train[ii]] = 1
y_train = y
y_train.shape

data shapes (7373, 32, 32, 1) (7373, 1) classes:  10
[0 1 2 3 4 5 6 7 8 9]


(7373, 10)

In [14]:
# split into training/validation
no_validation = int(0.1 * (x_train.shape[0]))

x_val = x_train[0:no_validation,...]
y_val = y_train[0:no_validation,...]

x_train = x_train[no_validation:,...]
y_train = y_train[no_validation:,...]

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

# define image generators with mild augmentation
train_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(x=x_train,\
                                     y=y_train,\
                                     batch_size=batch_size,\
                                     shuffle=True)

test_datagen = ImageDataGenerator()

val_generator = test_datagen.flow(x=x_val,\
                                    y=y_val,\
                                    batch_size=batch_size,\
                                    shuffle=True)

(6636, 32, 32, 1) (6636, 10) (737, 32, 32, 1) (737, 10)


In [20]:
# define model AlexNet (but topless)
model = Sequential()

model.add(Conv2D(filters=96, kernel_size=(5,5), strides=1,input_shape=(dim_x, dim_y, 1), activation=tf.nn.relu))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=1, activation=tf.nn.relu))
#model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=1, activation=tf.nn.relu))
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=1, activation=tf.nn.relu))
model.add(SpatialDropout2D(rate=0.67))
model.add(Conv2D(filters=250, kernel_size=(3,3), strides=1, activation=tf.nn.relu))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Reshape((1000,1)))
model.add(AveragePooling1D(pool_size=100,strides=100))
model.add(Reshape(([10])))
model.add(Activation(tf.nn.softmax))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 28, 28, 96)        2496      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 10, 10, 256)       614656    
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 8, 8, 384)         885120    
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 6, 6, 384)         1327488   
_________________________________________________________________
spatial_dropout2d_5 (Spatial (None, 6, 6, 384)         0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 4, 4, 250)         864250    
__________

In [21]:
def learning_schedule(epoch):
    if epoch <= 5:
        lr = 3e-4
    elif epoch <= 10:
        lr =1e-4
    elif epoch <= 50:
        lr = 1e-5
    elif epoch <= 150:
        lr = 1e-6
    else:
        lr = 1e-8
    return lr

# callbacks
lrate = LearningRateScheduler(learning_schedule)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=600, verbose=1, mode='auto')

In [23]:
steps_per_epoch = int(len(y_train)/batch_size)
max_epochs = 2048


history = model.fit_generator(generator=train_generator,\
                                steps_per_epoch=steps_per_epoch,\
                                validation_data=val_generator,\
                                validation_steps=50,\
                                epochs=max_epochs,\
                                callbacks=[early, lrate],\
                                verbose=1)

Epoch 1/2048
208/208 [==============================] - 172s 828ms/step - loss: 1.9198 - acc: 0.2364 - val_loss: 1.7692 - val_acc: 0.3134
Epoch 2/2048
208/208 [==============================] - 166s 796ms/step - loss: 1.6953 - acc: 0.3036 - val_loss: 1.6856 - val_acc: 0.3066
Epoch 3/2048
208/208 [==============================] - 156s 751ms/step - loss: 1.6211 - acc: 0.3214 - val_loss: 1.6879 - val_acc: 0.3189
Epoch 4/2048
208/208 [==============================] - 158s 757ms/step - loss: 1.5655 - acc: 0.3434 - val_loss: 1.6360 - val_acc: 0.3297
Epoch 5/2048
208/208 [==============================] - 148s 709ms/step - loss: 1.5028 - acc: 0.3737 - val_loss: 1.5383 - val_acc: 0.3446
Epoch 6/2048
208/208 [==============================] - 150s 719ms/step - loss: 1.4529 - acc: 0.3996 - val_loss: 1.5188 - val_acc: 0.3284
Epoch 7/2048
208/208 [==============================] - 164s 790ms/step - loss: 1.3504 - acc: 0.4376 - val_loss: 1.5050 - val_acc: 0.3392
Epoch 8/2048
208/208 [============

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(15,12))
plt.subplot(211)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title("Accuracy and Loss",fontsize=28)
plt.ylabel('accuracy',fontsize=24)
plt.legend(['Train','Val'],fontsize=18)

plt.subplot(212)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch',fontsize=24)
plt.ylabel('loss',fontsize=24)
plt.legend(['Train','Val'],fontsize=18)
plt.show()

In [ ]:
x_test = pd.read_csv('../input/test.csv')
x_test.head()

x_test = np.array(x_test.values)
x_test = x_test / 255.

print("data shape", x_test.shape)

x_test = x_test.reshape((-1, dim_x,dim_y,1))


In [ ]:
# predict!
y_pred = model.predict(x_test)

In [ ]:
# visualize success (?) :/

def imshow_w_labels(img,  pred,count):
    plt.subplot(1,4,count+1)
    plt.imshow(img, cmap="gray")
    plt.title("Prediction: %i, "%(pred), fontsize=24)
    
    
count = 0
mask = [1,3,3,7]
plt.figure(figsize=(24,6))
for kk in range(50,600):
    
    if y_pred[kk,:].argmax() == mask[count]:
        imshow_w_labels(x_test[kk,:,:,0],y_pred[kk,...].argmax(), count)
        count += 1
    if count >= 4: break
plt.show()

In [ ]:

# convert one-hot predictions to indices
results = np.argmax(y_pred,axis = 1)

results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("cnn_mnist_datagen.csv",index=False)